In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import re
import warnings
import datetime as dt # import datetime 
from datetime import datetime
import time

warnings.filterwarnings('ignore')

In [2]:
#list for retrieved data storage
link_list=list()
cat_list=list()
addr_list=list()
rge_list=list()
lotto_list=list()
base_list=list()
data_list=list()

In [3]:
#script customization
tr_code='0151460094'
tipo_bene='immobili'
cat='immobile_residenziale'

In [4]:
#build PVP link
def pvp_link_builder(tipo_bene,cat,tr_code,pagina):
    pvp_link_root='https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?'
    tipo_bene='tipo_bene={tipo_bene}'.format(tipo_bene=tipo_bene)
    categoria='categoria={cat}'.format(cat=cat)
    tribunale='tribunale={tr_code}'.format(tr_code=tr_code)
    ordinamento='ordinamento=data_vendita_decre'
    pagina='frame4_item={pagina}'.format(pagina=pagina)
    viz='&view=tab&elementiPerPagina=50'
    #overall param search string
    param_search='&'.join([tipo_bene,categoria,tribunale,ordinamento,pagina,viz])
    #final link
    pvp_link=''.join([pvp_link_root,param_search])
    return pvp_link

In [5]:
def pvp_link_get_content(pvp_link):
    #retrieve page content
    try:
        page=requests.get(pvp_link)
        print('http request response: '+str(page.status_code))
        print('overall execution time [seconds]: '+ str(page.elapsed.total_seconds()))
        return page
    except requests.exceptions.RequestException as e: 
        print('connection Error')
        raise SystemExit(e)

In [6]:
def pvp_link_analyze(page):
    #list for retrieved data storage (local)
    link_list=list()
    cat_list=list()
    addr_list=list()
    rge_list=list()
    lotto_list=list()
    base_list=list()
    data_list=list()
    soup=BeautifulSoup(page.content,'html.parser')
    imm_list=soup.find_all('div',class_='col-md-6 col-lg-4 col-sm-6 col-xs-12 tile-dettaglio')
    for rge in imm_list:
        if rge.find('span',class_='font-black inline') is not None:
            link_list.append(rge.find('a')['href'])
            cat_list.append(rge.find('span',class_='categoria').get_text(strip=True))
            addr_list.append(rge.find('div',class_='anagrafica-risultato').get_text(strip=True))
            base_list.append(rge.find('span',class_='font-blue font18').get_text(strip=True))
            rge_list.append(rge.find('span',class_='font-black inline').get_text(strip=True))
            data_list.append(rge.find('span',class_='font-green inline').get_text(strip=True))
            lotto_list.append(rge.find('span',class_='black').get_text(strip=True))

    allField_list=[link_list,cat_list,addr_list,rge_list,lotto_list,base_list,data_list]
    return allField_list

In [7]:
#run condition setup
time_step=dt.timedelta(days=60)
date_ref=datetime.now()-time_step
###########################################
#start from 1st page for the 1st program run
if len(data_list)==0:
    pagina=1
    pvp_link=pvp_link_builder(tipo_bene,cat,tr_code,pagina)
    print("start")
    print(pvp_link)
    page=pvp_link_get_content(pvp_link)
    allField_list=pvp_link_analyze(page)
    #append to global list
    link_list.extend(allField_list[0])
    cat_list.extend(allField_list[1])
    addr_list.extend(allField_list[2])
    rge_list.extend(allField_list[3])
    lotto_list.extend(allField_list[4])
    base_list.extend(allField_list[5])
    data_list.extend(allField_list[6])
    
    

if len(data_list)>0:
    date_check = datetime.strptime(data_list[-1], '%d/%m/%Y %H:%M')
    gap=date_check-date_ref
    print(gap)
    while gap>dt.timedelta(days=0):
        pagina+=1
        pvp_link=pvp_link_builder(tipo_bene,cat,tr_code,pagina)
        print("loop")
        print("link:\n"+pvp_link+"\npag:\n"+str(pagina))
        time.sleep(30)
        page=pvp_link_get_content(pvp_link)
        allField_list=pvp_link_analyze(page)
        #append to global list
        link_list.extend(allField_list[0])
        cat_list.extend(allField_list[1])
        addr_list.extend(allField_list[2])
        rge_list.extend(allField_list[3])
        lotto_list.extend(allField_list[4])
        base_list.extend(allField_list[5])
        data_list.extend(allField_list[6])
        
        date_check = datetime.strptime(data_list[-1], '%d/%m/%Y %H:%M')
        gap=date_check-date_ref
        print(gap)



start
https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?tipo_bene=immobili&categoria=immobile_residenziale&tribunale=0151460094&ordinamento=data_vendita_decre&frame4_item=1&&view=tab&elementiPerPagina=50
http request response: 200
overall execution time [seconds]: 5.640455
146 days, 3:36:35.575129
loop
link:
https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?tipo_bene=immobili&categoria=immobile_residenziale&tribunale=0151460094&ordinamento=data_vendita_decre&frame4_item=2&&view=tab&elementiPerPagina=50
pag:
2
http request response: 200
overall execution time [seconds]: 5.210072
100 days, 19:36:35.575129
loop
link:
https://pvp.giustizia.it/pvp/it/risultati_ricerca.page?tipo_bene=immobili&categoria=immobile_residenziale&tribunale=0151460094&ordinamento=data_vendita_decre&frame4_item=3&&view=tab&elementiPerPagina=50
pag:
3
http request response: 200
overall execution time [seconds]: 6.724034
95 days, 19:36:35.575129
loop
link:
https://pvp.giustizia.it/pvp/it/risultati_ricerca.pa

In [8]:
#data preprocessing
full_link_list=['http://pvp.giustizia.it'+sub for sub in link_list]  
cat_list=[x.upper() for x in cat_list]
addr_list=[x.upper() for x in addr_list]
base_list=[x.upper() for x in base_list]
base_list=[re.sub(r'[\n \r \s]|LOTTO|NR\.|\€','',base) for base in base_list]
rge_list=[x.upper() for x in rge_list]
data_list=[x.upper() for x in data_list]
lotto_list=[x.upper() for x in lotto_list]
lotto_list=[re.sub(r'[\n \r \s]|LOTTO|NR\.','',lotto) for lotto in lotto_list]

In [9]:
#pvp_info_write()
info_dict={'RGE':rge_list,'LOTTO':lotto_list,'INDIRIZZO':addr_list,\
'CATEGORIA':cat_list,'DATA':data_list,\
'BASE_ASTA':base_list,'LINK':full_link_list}

df=pd.DataFrame(data=info_dict)
outFile=('PVP_MILANO_{0}_{1}_{2}.csv').format(dt.datetime.today().year,dt.datetime.today().month,dt.datetime.today().day)
df.to_csv(outFile,index=False)